# MLCommons Earthquake GPU Time Analysis
- Generates Interactive plot of experiments and execution time
- Create custom plots for experiments of interest

In [ ]:
import os
import re
import glob
import pickle
import datetime
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import timedelta
import matplotlib.dates as md
import matplotlib.pyplot as plt
from bokeh.palettes import Spectral4, Category20, Spectral11, Category20_20
from bokeh.plotting import figure, output_file, show
from bokeh.models import Legend
from bokeh.io import output_notebook

In [ ]:
def plot_train_times(data_dict, path, gpu=None, timer='RunTFTCustomVersion train', title=None, sys=True, plot_dict=False):
    """ Create plot to compare training times and save figure.
    Args:
        data_dict: dictionary of run data. 
        path: output path.
    """
    fig = plt.figure(figsize=(8, 6), dpi=360)
    time_dir = os.path.join(path,'train_times','static')
    

    if sys == False:
        data = pd.DataFrame(columns=['gpu','epochs','time'])
        for experiment in data_dict.keys():
            if not plot_dict is False:
                if not experiment in plot_dict.keys():
                    continue
            row = pd.Series(dtype='object')            
            timer_df = data_dict[experiment]['timer_df']
            if timer in timer_df['timer'].values:
                row['time'] = float(timer_df.loc[timer_df['timer'] == timer]['time'].values[0])/3600
                row['gpu'] = data_dict[experiment]['run_info']['gpu']
                row['epochs'] = int(data_dict[experiment]['run_info']['epochs'])
                data = pd.concat([data, row.to_frame().T])
            else:
                print(f"Missing train timer for {experiment}")

        # update dtypes
        data['epochs'] = data['epochs'].astype(int)
        data['time'] = data['time'].astype(float)
        hue = 'gpu'
        timer_ = timer.replace(' ','_')
        #save_name = os.path.join(time_dir,f'{timer_}_times.png')
        save_name = f'{timer_}_times'
        
    else:
        data = pd.DataFrame()
        for experiment in data_dict.keys():
            run = {}
            run['gpu'] = data_dict[experiment]['run_info']['gpu']
            run['system'] = data_dict[experiment]['run_info']['system']
            run['filesystem'] = data_dict[experiment]['run_info']['filesystem']
            run['epochs'] = int(data_dict[experiment]['run_info']['epochs'])
            run['date'] = data_dict[experiment]['run_info']['date']
            timer_df = data_dict[experiment]['timer_df']
            if timer in timer_df['timer'].values:
                run['time'] = float(timer_df[timer_df['timer'] == timer]['time'].values[0])/3600
                run['experiment'] = f"{run['gpu']}-{run['system']}-{run['filesystem']}-{run['date']}"
                if not plot_dict is False:
                    if not run['experiment'] in plot_dict.keys():
                        continue                 
                    run['experiment'] = plot_dict[run['experiment']]
                run_df = pd.DataFrame([run])
                data = pd.concat([data, run_df], ignore_index=True) 
        data = data[~data['experiment'].str.contains('rivanna-rivanna')]
        if gpu is not None:
            data = data[data['gpu']== gpu]
        hue = 'experiment'
        timer = timer.replace(' ','_')
        save_name = f'{gpu}_{timer}_times'
    
    # create plot
    sns.lineplot(x='epochs', 
                 y='time',
                 hue=hue,
                 data=data,
                 legend = False)
    sns.scatterplot(x='epochs', 
                    y='time',
                    hue=hue,
                    s=50,
                    data=data)
    
    epoch_list = data['epochs'].unique()
    if title is not None:
        plt.title(title)
        save_name = title
    else:
        plt.title(f'{timer} Time Comparison')
    plt.ylabel('Time (hours)')
    sns.set_style("whitegrid")
    if gpu == None and sys == True:
        plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
    if not os.path.exists(time_dir):
        os.makedirs(time_dir, exist_ok=True)
    plot, _ = SAVEFIG(fig, save_name, time_dir)
    if not plot_dict is False:
        plt.show()
    plt.close(fig)

In [ ]:
def plot_interactive_train_times(data_dict, path, gpu=None, timer='RunTFTCustomVersion train', title=None, sys=True):
    """ Create plot to compare training times and save figure.
    Args:
        data_dict: dictionary of run data. 
        path: output path.
    """
    p = figure(width=1000, height=600)
    time_dir = os.path.join(path,'train_times', 'interactive')
    colors = itertools.cycle(Category20_20)
    output_notebook()

    if sys == False:
        data = pd.DataFrame(columns=['gpu','epochs','time'])
        for experiment in data_dict.keys():
            row = pd.Series(dtype='object')            
            timer_df = data_dict[experiment]['timer_df']
            if timer in timer_df['timer'].values:
                row['time'] = float(timer_df.loc[timer_df['timer'] == timer]['time'].values[0])/3600
                row['gpu'] = data_dict[experiment]['run_info']['gpu']
                row['epochs'] = int(data_dict[experiment]['run_info']['epochs'])
                data = pd.concat([data, row.to_frame().T])
            else:
                print(f"Missing train timer for {experiment}")

        # update dtypes
        data['epochs'] = data['epochs'].astype(int)
        data['time'] = data['time'].astype(float)
        group = 'gpu'
        timer_ = timer.replace(' ','_')
        save_name = f'{timer_}_times'
        
    else:
        data = pd.DataFrame()
        for experiment in data_dict.keys():
            run = {}
            run['gpu'] = data_dict[experiment]['run_info']['gpu']
            run['system'] = data_dict[experiment]['run_info']['system']
            run['filesystem'] = data_dict[experiment]['run_info']['filesystem']
            run['epochs'] = int(data_dict[experiment]['run_info']['epochs'])
            run['date'] = data_dict[experiment]['run_info']['date']
            timer_df = data_dict[experiment]['timer_df']
            if timer in timer_df['timer'].values:
                run['time'] = float(timer_df[timer_df['timer'] == timer]['time'].values[0])/3600
                run['sys'] = f"{run['gpu']}-{run['system']}-{run['filesystem']}-{run['date']}"
                run_df = pd.DataFrame([run])
                data = pd.concat([data, run_df], ignore_index=True)        
        data = data[~data['sys'].str.contains('rivanna-rivanna')]
        if gpu is not None:
            data = data[data['gpu']== gpu]
        group = 'sys'
        timer_ = timer.replace(' ','_')
        save_name = f'{gpu}_{timer_}_times'   

    for item, color in zip(data[group].unique(), colors):
        filtered = data[data[group] == item].sort_values('epochs')
        p.line(filtered['epochs'], filtered['time'], line_width=2, color=color, alpha=0.8, legend_label=item)

    p.legend.location = 'top_right'
    p.legend.orientation = "vertical"
    p.legend.click_policy="hide"

    new_legend = p.legend[0]
    p.legend[0] = None
    p.add_layout(new_legend, 'right')
    
    if not os.path.exists(time_dir):
        os.makedirs(time_dir, exist_ok=True)
    
    output_file(os.path.join(time_dir,f"{timer_}_times_interactive.html"), title="interactive_legend.py example")
    show(p)

In [ ]:
def SAVEFIG(fig, filename, path=None, formats=('png', 'pdf')):
    fileout = os.path.join(path,filename)
    for my_format in formats:
        fig.savefig(f"{fileout}.{my_format}", format=my_format, bbox_inches="tight")
    return 1, tuple(f'{fileout}.{fmt}' for fmt in formats)

### Load Pickle

In [ ]:
cwd = os.getcwd()
pickle_file = os.path.join(cwd,'experiment_data.pkl')
with open(pickle_file, 'rb') as f:
    loaded_dict = pickle.load(f)

### Display Available Data

In [ ]:
data = pd.DataFrame()
for experiment in loaded_dict.keys():
    run = {}
    gpu = loaded_dict[experiment]['run_info']['gpu']
    system = loaded_dict[experiment]['run_info']['system']
    filesystem = loaded_dict[experiment]['run_info']['filesystem']
    run['count'] = 1
    run['sys'] = f"{gpu}-{system}-{filesystem}"
    run_df = pd.DataFrame([run])
    data = pd.concat([data, run_df], ignore_index=True) 
data.groupby('sys').count()

### Create Analysis Outputs

In [ ]:
analysis_path = os.path.join(os.getcwd(),'analysis')
#train time plots
plot_train_times(loaded_dict, analysis_path, sys=False)
gpus = []
for experiment in loaded_dict.keys():
    gpu = loaded_dict[experiment]['run_info']['gpu']
    if not gpu in gpus:
        gpus.append(gpu)
for gpu in gpus:
    plot_train_times(loaded_dict, analysis_path, gpu=gpu, timer='total', sys=True)
    plot_train_times(loaded_dict, analysis_path, gpu=gpu, timer='RunTFTCustomVersion train', sys=True)

# Interactive Plots

In [ ]:
# train time
plot_interactive_train_times(loaded_dict, analysis_path, timer='RunTFTCustomVersion train', sys=True)

In [ ]:
# total time
plot_interactive_train_times(loaded_dict, analysis_path, timer='total', sys=True)

### Create Custom Plots
- Select experiments to plot from table displayed below
- Create a dictionary with experiment name as the key and display name as the values
- Feed dictionary as an argument to plot_train_times as the 'plot_dict' argument

#### Show available data to plot

In [ ]:
available = []
for experiment in loaded_dict.keys():
    run = {}
    run['gpu'] = loaded_dict[experiment]['run_info']['gpu']
    run['system'] = loaded_dict[experiment]['run_info']['system']
    run['filesystem'] = loaded_dict[experiment]['run_info']['filesystem']
    run['epochs'] = int(loaded_dict[experiment]['run_info']['epochs'])
    run['date'] = loaded_dict[experiment]['run_info']['date']
    #timer_df = loaded_dict[experiment]['timer_df']
    #if timer in timer_df['timer'].values:
    #run['time'] = float(timer_df[timer_df['timer'] == timer]['time'].values[0])/3600
    run['sys'] = f"{run['gpu']}-{run['system']}-{run['filesystem']}-{run['date']}"
    available.append(run['sys'])
available = set(available)
df = pd.DataFrame(available)
df.columns = ['experiment']
df

In [ ]:
plot_dict = {'v100-rivanna-project-2022-05-26':'V100 DFS',
             'v100-rivanna-shm-2022-07-21':'V100 Shared Memory'}

In [ ]:
plot_train_times(loaded_dict, analysis_path, gpu=None, timer='total', sys=True, plot_dict=plot_dict)